In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
print(np.__version__)

C:\Users\luiho\miniconda3\envs\LoLa_NLI_BERT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.26.4


In [2]:
!where python

C:\Users\luiho\AppData\Local\Programs\Python\Python311\python.exe
C:\msys64\mingw64\bin\python.exe
C:\Users\luiho\AppData\Local\Programs\Python\Python39\python.exe
C:\Users\luiho\AppData\Local\Microsoft\WindowsApps\python.exe


In [4]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
# Step 1: Load the SNLI dataset
snli = load_dataset("snli")

# Filter to remove invalid rows (-1 labels)
for split in snli:
    snli[split] = snli[split].filter(lambda x: x["label"] >= 0)


In [6]:
#Load the tokenizer and model
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

# Tokenize the dataset
encoded_snli = snli.map(tokenize_function, batched=True)

Map: 100%|██████████| 9842/9842 [00:00<00:00, 14248.77 examples/s]


In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

C:\Users\luiho\miniconda3\envs\LoLa_NLI_BERT\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_snli["train"].select(range(50000)),  # Use a subset of 50k samples for training
    eval_dataset=encoded_snli["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\luiho\AppData\Local\Temp\ipykernel_13204\764375851.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.530000,0.482097,0.816196,0.815902,0.815726,0.816196


TrainOutput(global_step=3125, training_loss=0.6171725244140625, metrics={'train_runtime': 315.1943, 'train_samples_per_second': 158.632, 'train_steps_per_second': 9.915, 'total_flos': 560305692331200.0, 'train_loss': 0.6171725244140625, 'epoch': 1.0})

In [13]:

results = trainer.evaluate()
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 0.48209720849990845, 'eval_accuracy': 0.8161958951432635, 'eval_f1': 0.8159016319195141, 'eval_precision': 0.8157262736094275, 'eval_recall': 0.8161958951432635, 'eval_runtime': 11.8015, 'eval_samples_per_second': 833.961, 'eval_steps_per_second': 52.197, 'epoch': 1.0}
